# Reconstructing a <i>Mimulus naiandinus</i> genome from hybrid genomes

## <a id="contents">Table o' contents</a>

[Look at read qualities](#read_quality)

[Align our reads to the reference genome](#align)

[Filtering and Processing alignments](#FandP)
  - [Convert SAMs to BAMs](#sam2bam)
  - [Reorder alignments to reference](#reorder)
  - [DeDuplicate reads](#dedupe)
  - [Merging BAM files](#mergeBAM)
  - [Indices and dictionaries](#inDic)
  - [Validate Merged BAM file](#validateSam)
  
[Calling variants](#callVars)

[Construct a draft <i>M. naiandius </i> genome](#altGenome)

[Making a map of genome coverage](#gencov)
  - [Make overall coverage map](#bigCov)
  - [Subset coverage map to scaffolds of interest](#subScaff)
  
[Subset <i>M. naiandinus</i> genome to just scaffolds of interest](#subGen)

[Spot check: polymorphism in pla2](#spotcheck)

## <a id="read_quality">Look at read qualities</a>

Let's use fastx tools to take a look at the average read qualities.

In [6]:
## path to our fastq files
fpath='/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/'

Fastx requires that we calculate the statistics first:

In [5]:
for i in $fpath*"fastq"
do
echo $i
$ftd"fastx_quality_stats" -Q33 -i $i -o $i"-Stats.txt"
echo $i done!
done


/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S1_S16_L002_R1_001.fastq
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S1_S16_L002_R1_001.fastq done!
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S2_S17_L002_R1_001.fastq
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S2_S17_L002_R1_001.fastq done!
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S3_S18_L002_R1_001.fastq
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S3_S18_L002_R1_001.fastq done!
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S4_S19_L002_R1_001.fastq
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S4_S19_L002_R1_001.fastq done!
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S5_S20_L002_R1_001.fastq
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S5_S20_L002

Then graph:

In [8]:
for i in $fpath*Stats.txt; do
fastq_quality_boxplot_graph.sh -i $i -o $i"quality.eps" -p
echo $i"quality.eps"
done


/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S1_S16_L002_R1_001.fastq-Stats.txtquality.eps
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S2_S17_L002_R1_001.fastq-Stats.txtquality.eps
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S3_S18_L002_R1_001.fastq-Stats.txtquality.eps
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S4_S19_L002_R1_001.fastq-Stats.txtquality.eps
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S5_S20_L002_R1_001.fastq-Stats.txtquality.eps
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S6_S21_L002_R1_001.fastq-Stats.txtquality.eps
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/merged.fastq-Stats.txtquality.eps


What do these look like? Here is our merged file, that has all of the reads:

![read_quality](merged.fastq-Stats.txtquality.png)

Looks great, high quality reads, with almost no variance. 

## <a id="align">Align our reads to the reference genome</a>

We'll use [bowtie2](http://bowtie-bio.sourceforge.net/bowtie2/index.shtml) to align our reads to our reference genome. The reference genome needs to be indexed in the way that bowtie2 wants, first:

In [10]:
bowtie2-build -f /Users/danthomas/Documents/naiandinus_gen/Luteus_genome/Mimulus_luteus.fasta Mlu

Settings:
  Output files: "Mlu.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  /Users/danthomas/Documents/naiandinus_gen/Luteus_genome/Mimulus_luteus.fasta
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:09
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:04
bmax according to bmaxDivN setting: 98519930
Using parameters --bmax 73889948 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these paramet

Then we can loop through our fastq files and do alignments against this reference genome. For the moment, we'll do these as individual alignments of the six bins of phenotypes assigned by Arielle and Josh. This results in a sequence alignment map (SAM) for each of the six.

In [11]:
cd /Users/danthomas/Documents/naiandinus_gen/Luteus_genome

for i in $fpath*001.fastq; do
    j=$(basename $i)
    k=${j/fastq/sam}
    bowtie2 --local  -x Mlu -U $i -S /Users/danthomas/Documents/naiandinus_gen/SAM/$k \
done

Some output:

In [23]:
cat bowtie_log.txt

56034871 reads; of these:
  56034871 (100.00%) were unpaired; of these:
    7893494 (14.09%) aligned 0 times
    11729089 (20.93%) aligned exactly 1 time
    36412288 (64.98%) aligned >1 times
85.91% overall alignment rate
68049510 reads; of these:
  68049510 (100.00%) were unpaired; of these:
    7592098 (11.16%) aligned 0 times
    14879687 (21.87%) aligned exactly 1 time
    45577725 (66.98%) aligned >1 times
88.84% overall alignment rate
60681218 reads; of these:
  60681218 (100.00%) were unpaired; of these:
    9222716 (15.20%) aligned 0 times
    12679236 (20.89%) aligned exactly 1 time
    38779266 (63.91%) aligned >1 times
84.80% overall alignment rate
55885597 reads; of these:
  55885597 (100.00%) were unpaired; of these:
    7485075 (13.39%) aligned 0 times
    11651355 (20.85%) aligned exactly 1 time
    36749167 (65.76%) aligned >1 times
86.61% overall alignment rate
67627796 reads; of these:
  67627796 (100.00%) were unpaired; of these:
    15149703 (22.40%) aligned 0 time

Time for these alignments, as a whole:

real    565m3.770s  
user    554m22.225s  
sys     4m27.857s


These SAM files look like this, a header that lists out the lengths of our scaffolds, and a body that is a long list of where the various reads were assigned on the reference genome, with a mapq quality score for the confidence in the placement of this read on the reference genome, the quality of the basecalls, other good stuff. More info <a href="https://en.wikipedia.org/wiki/SAM_(file_format)">here</a>.

In [17]:
head JP-S6_S21_L002_R1_001.sam

@HD	VN:1.0	SO:unsorted
@SQ	SN:lcl|scaffold_32	LN:817265
@SQ	SN:lcl|scaffold_21	LN:849581
@SQ	SN:lcl|scaffold_14	LN:884134
@SQ	SN:lcl|scaffold_37	LN:783220
@SQ	SN:lcl|scaffold_23	LN:851886
@SQ	SN:lcl|scaffold_8	LN:980590
@SQ	SN:lcl|scaffold_9	LN:976062
@SQ	SN:lcl|scaffold_33	LN:797088
@SQ	SN:lcl|scaffold_38	LN:791999


In [18]:
tail JP-S6_S21_L002_R1_001.sam

K00282:27:HC3GVBBXX:2:2228:30959:49194	0	lcl|scaffold_606	126980	44	48M3S	*	0	0	TTGAGCCTTTTGTAGTAGGATTTATGAATTTCTCTGGAGCTTTATTAGACT	AAFFFJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJ	AS:i:96	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:48	YT:Z:UU
K00282:27:HC3GVBBXX:2:2228:31081:49194	16	lcl|scaffold_24	610319	11	1S33M1I14M2S	*	0	0	CCATGTTGGAGTCCGAGTTTTGGAATGTTGCATGAAATAAGAATTGTAGAC	JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJFFFAA	AS:i:78	XS:i:76	XN:i:0	XM:i:1	XO:i:1	XG:i:1	NM:i:2	MD:Z:30G16	YT:Z:UU
K00282:27:HC3GVBBXX:2:2228:31284:49194	16	lcl|scaffold_1046	13016	17	51M	*	0	0	CCGCGATTTCTCTCGATTTCTAATGCAATACACAAATCAATTGGTTCTGTT	JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJFFFAA	AS:i:94	XS:i:80	XN:i:0	XM:i:1	XO:i:0	XG:i:0	NM:i:1	MD:Z:24T26	YT:Z:UU
K00282:27:HC3GVBBXX:2:2228:31527:49194	0	lcl|scaffold_448	249337	31	51M	*	0	0	AGTAGCATGGGCTGCAATAAAGGCCCGGTGTCATTGTGTTAATAAAAAATG	AAFFFJJJJJJJJJJJJ<JJJJJJJJJJJJJJFJJJJJJJJJJJJJJJJJJ	AS:i:102	XS:i:94	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:51	Y

## <a id="FandP">Filtering and Processing alignments</a>

Before calling our variants (places where our new genomes are different from the reference), we've got some cleanup to do to our new alignments.

### <a id="sam2bam">Convert SAMs to BAMs</a>

For downstream work, we'll work with compressed SAM files, called BAM files. We'll use <a href="http://www.htslib.org">Samtools</a> for this and other tasks.

In [25]:
cd /Users/danthomas/Documents/naiandinus_gen/SAM

for i in *sam; do
    time samtools view $i -bS > ./BAM/${i/sam/bam} && \
    echo $i
done

JP-S1_S16_L002_R1_001.sam
JP-S2_S17_L002_R1_001.sam
JP-S3_S18_L002_R1_001.sam
JP-S4_S19_L002_R1_001.sam
JP-S5_S20_L002_R1_001.sam
JP-S6_S21_L002_R1_001.sam


## <a id="reorder">Reorder alignments to reference</a>

It's also necessary to sort the alignments in the order they appear on the reference genome. <a href="https://broadinstitute.github.io/picard/">Picard</a> tools are useful here. 

In [ ]:
cd /Users/danthomas/Documents/naiandinus_gen/BAM

for i in *
do 
    output="sortedBAM/"${i/.bam/.sorted.bam}
    picard SortSam I=$i O=$output SORT_ORDER=coordinate && \
    ls $i
done

## <a id="dedupe">DeDuplicate reads</a>

We can also use Picard tools to remove duplicate reads. This has been recommended to me, and it's discussed <a href="https://broadinstitute.github.io/picard/command-line-overview.html#MarkDuplicates">here</a>. 

I'm a little unclear on this. I can see why removing duplicates is handy.  Dereplicating the redundant information in duplicates makes sense as far as computational expense. But the explanation in picard's manual implies that all duplicates are erroroneous reads, either from bench preps or the cameras of the sequencers - how can this be true? Presumably the same digestion site occurs several times, gets sequenced? Maybe the sites of digestion are random, so it is really unlikely that a sequence would happen twice. If duplicate reads are real (can they be?) aren't we losing converage by removing them?

Not sure, anyway, just following orders like a good scientist...

In [ ]:
## home for our deduplicated maps:
ddm="/Users/danthomas/Documents/naiandinus_gen/BAM/dmBAM/"

cd /Users/danthomas/Documents/naiandinus_gen/BAM/sortedBAM

for i in *
do
    output=${i/.sorted.bam/.MD.bam}
    echo $output
    picard MarkDuplicates \
        I=$i \
        O=$ddm$output \
        M=$output.metrics.txt \
        REMOVE_DUPLICATES=true \
        MAX_FILE_HANDLES_FOR_READ_ENDS_MAP=256 && echo $i
done


We got the last input by asking our computer what kind of load it can handle at the moment, in terms of number of files:

In [28]:
ulimit -n 

256


### <a id="readGroups">Adding read group information to reads</a>

We're going to merge all of 6 of the genomes, that were binned by phenotype, for the variant calls. Before we do this, we want to retain the information about what phenotype a given read came from. In the fastq files and SAM/BAM files, this information is encoded in the read name. But from here this information is tracked as part of the "RG" info, or read group information, in the merged BAM file. Picard tools again:

In [ ]:
cd /Users/danthomas/Documents/naiandinus_gen/BAM/dmBAM

rgBAMs="/Users/danthomas/Documents/naiandinus_gen/BAM/rgBAM/"

touch $rgBAMs"rglog.txt"

for i in *; do
    echo $i >> $rgBAMs"rglog.txt"
    SM=$(echo $i | sed "s/.*_S/S/g" | sed "s/_L.*//g")
    output=$rgBAMs${i/.MD.bam/.RG.bam}
    picard AddOrReplaceReadGroups  I=$i  O=$output  RGPL=illumina  RGPU=indexSEQ  RGID=27  RGLB=A_Library  RGSM=$SM
    echo $output >> $rgBAMs"rglog.txt"
done


### <a id="mergeBAM">Merging BAM files</a>

Let's put these six genomes together into a single alignment, which we'll then use to look for areas of difference between our pooled F2 hybrids and the <i>Mimulus luteus</i> genome. 

In [1]:
cd /Users/danthomas/Documents/naiandinus_gen/BAM/rgBAM

for i in *; do
    ins="$ins $i"
done

#echo $ins

samtools merge Mll_x_MN.merged.bam $ins

### <a id="inDic">Indices and dictionaries</a>

We're almost ready to call the variants. The final step is create a Samtools index for both our merged BAM file and our reference genome, and create a dictionary of reference sequences using Picard tools:

In [ ]:
## shortcut to reference genome fasta
genome="/Users/danthomas/Documents/naiandinus_gen/Luteus_genome/Mimulus_luteus.fasta"

cd /Users/danthomas/Documents/naiandinus_gen/Luteus_genome/

## make dictionary of reference sequences
picard CreateSequenceDictionary REFERENCE=$genome OUTPUT=Mimulus_luteus.dict

## use make a samtools-type index
samtools faidx Mimulus_luteus.fasta

faidx="/Users/danthomas/Documents/naiandinus_gen/Luteus_genome/Mimulus_luteus.fasta.fai"

## index our bam file

cd /Users/danthomas/Documents/naiandinus_gen/BAM/

samtools index /Users/danthomas/Documents/naiandinus_gen/BAM/Mll_x_MN.merged.bam

### <a id="validateSam">Validate Merged BAM file</a>

We can check to make sure that we haven't screwed things up too badly with Picard's <a href="https://broadinstitute.github.io/picard/command-line-overview.html#ValidateSamFile">ValidateSamFile</a>:

In [2]:
## shortcut to the merged BAM file:
merged="/Users/danthomas/Documents/naiandinus_gen/BAM/Mll_x_MN.merged.bam"

picard ValidateSamFile I=$merged MODE=SUMMARY

15:05:56.235 INFO  NativeLibraryLoader - Loading libgkl_compression.dylib from jar:file:/usr/local/Cellar/picard-tools/2.18.2/libexec/picard.jar!/com/intel/gkl/native/libgkl_compression.dylib
[Wed May 02 15:05:56 PDT 2018] ValidateSamFile INPUT=/Users/danthomas/Documents/naiandinus_gen/BAM/Mll_x_MN.merged.bam MODE=SUMMARY    MAX_OUTPUT=100 IGNORE_WARNINGS=false VALIDATE_INDEX=true INDEX_VALIDATION_STRINGENCY=EXHAUSTIVE IS_BISULFITE_SEQUENCED=false MAX_OPEN_TEMP_FILES=8000 SKIP_MATE_VALIDATION=false VERBOSITY=INFO QUIET=false VALIDATION_STRINGENCY=STRICT COMPRESSION_LEVEL=5 MAX_RECORDS_IN_RAM=500000 CREATE_INDEX=false CREATE_MD5_FILE=false GA4GH_CLIENT_SECRETS=client_secrets.json USE_JDK_DEFLATER=false USE_JDK_INFLATER=false
[Wed May 02 15:05:56 PDT 2018] Executing as danthomas@wks-14-199 on Mac OS X 10.12.6 x86_64; Java HotSpot(TM) 64-Bit Server VM 1.8.0_144-b01; Deflater: Intel; Inflater: Intel; Provider GCS is not available; Picard version: 2.18.2-SNAPSHOT
INFO	2018-05-02 15:06:22	Sa

No errors found, onward.

## <a id="callVars">Calling variants</a>

So now we're to examine differences between our alignment of our hybrid genomes and the reference genome. We'll do this with the <a href="https://software.broadinstitute.org/gatk/documentation/tooldocs/current/org_broadinstitute_hellbender_tools_walkers_haplotypecaller_HaplotypeCaller.php">HaplotypeCaller</a> from the <a href="https://software.broadinstitute.org/gatk/">GATK</a> toolset. 

For the moment, we'll use the default settings:

In [ ]:
genome="/Users/danthomas/Documents/naiandinus_gen/Luteus_genome/Mimulus_luteus.fasta"
merged="/Users/danthomas/Documents/naiandinus_gen/BAM/Mll_x_MN.merged.bam"
out="/Users/danthomas/Documents/naiandinus_gen/Mll_x_MN.merged.ver0.1.vcf"


GATK HaplotypeCaller \
    -R $genome \
    -I $merged \
    -O $out

So this gives a variant call file (.vcf), which is a list of all the spots where the haplotyper found what it considers statistically significant differences between the BAM alignment we constructed from our reads, and the reference genome that we used for the alignment. 

The headers have information on the settings and the formats of the output, and a list of scaffolds. Looks like this:

In [4]:
vcf="/Users/danthomas/Documents/naiandinus_gen/Mll_x_MN.merged.ver0.1.vcf"
head $vcf

##fileformat=VCFv4.2
##FILTER=<ID=LowQual,Description="Low quality">
##FORMAT=<ID=AD,Number=R,Type=Integer,Description="Allelic depths for the ref and alt alleles in the order listed">
##FORMAT=<ID=DP,Number=1,Type=Integer,Description="Approximate read depth (reads with MQ=255 or with bad mates are filtered)">
##FORMAT=<ID=GQ,Number=1,Type=Integer,Description="Genotype Quality">
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
##FORMAT=<ID=PL,Number=G,Type=Integer,Description="Normalized, Phred-scaled likelihoods for genotypes as defined in the VCF specification">
##GATKCommandLine=<ID=HaplotypeCaller,CommandLine="HaplotypeCaller  --output /Users/danthomas/Documents/naiandinus_gen/Mll_x_MN.merged.ver0.1.vcf --input /Users/danthomas/Documents/naiandinus_gen/BAM/Mll_x_MN.merged.bam --reference /Users/danthomas/Documents/naiandinus_gen/Luteus_genome/Mimulus_luteus.fasta  --annotation-group StandardAnnotation --annotation-group StandardHCAnnotation --disable-tool-default-annota

The rest of the document is a list of differences. The GATK folks do a good job on explaining how to read these <a href="https://gatkforums.broadinstitute.org/gatk/discussion/1268/what-is-a-vcf-and-how-should-i-interpret-it">here</a>. 

In [5]:
grep CHROM $vcf -A 5 -B 5

##contig=<ID=lcl|scaffold_35,length=785883>
##contig=<ID=lcl|scaffold_896,length=153320>
##contig=<ID=lcl|scaffold_280,length=352622>
##contig=<ID=lcl|scaffold_1160,length=116697>
##source=HaplotypeCaller
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	S16	S17	S18	S19	S20	S21
lcl|scaffold_32	209	.	G	A	210.56	.	AC=5;AF=0.417;AN=12;BaseQRankSum=0.340;ClippingRankSum=0.000;DP=20;ExcessHet=9.1645;FS=2.171;MLEAC=5;MLEAF=0.417;MQ=44.00;MQRankSum=0.000;QD=26.32;ReadPosRankSum=0.996;SOR=0.528	GT:AD:DP:GQ:PL	0/1:1,1:2:39:39,0,40	0/0:2,0:2:6:0,6,92	0/1:3,1:4:33:33,0,106	0/1:3,1:4:33:33,0,124	0/1:1,2:3:33:81,0,33	0/1:3,2:5:62:62,0,118
lcl|scaffold_32	340	.	T	C	285.29	.	AC=6;AF=0.600;AN=10;BaseQRankSum=0.728;ClippingRankSum=0.000;DP=15;ExcessHet=7.2016;FS=0.000;MLEAC=6;MLEAF=0.600;MQ=44.00;MQRankSum=0.000;QD=25.36;ReadPosRankSum=-0.642;SOR=1.061	GT:AD:DP:GQ:PL	1/1:0,2:2:6:90,6,0	0/1:2,1:3:36:36,0,81	./.	0/1:1,3:4:24:123,0,24	0/1:1,1:2:38:38,0,39	0/1:3,1:4:33:33,0,123
lcl|scaffold_32	385	.	C	A	162.28

## <a id="altGenome">Construct a draft <i>M. naiandius </i> genome</a>

We now want to take these differences that we've found, and place them onto the skeleton of the reference genome. This is a less than perfect method, it assumes that everywhere a statistically significant difference was not found, the <i> M. naiandinus</i> has the same sequence info as the reference genome of <i>M. luteus</i>. More discussion on this below. 

We'll use [FastaAlternateReferenceMaker](https://software.broadinstitute.org/gatk/documentation/tooldocs/3.8-0/org_broadinstitute_gatk_tools_walkers_fasta_FastaAlternateReferenceMaker.php), a tool from GATK. From cruising the forums it seems like this is the most popular tool for this kind of thing, but it has not yet been ported over to the newest version of the GATK tools (4.x). It was in the last generation of GATK 3.8 tools, so I had install it from their [archives](https://software.broadinstitute.org/gatk/download/archive). 

In [6]:
## symlink to the jar file of GATK 3.8, cuz I have two GATK installs now
ln -s /usr/local/Cellar/GenomeAnalysisTK-3.8/GenomeAnalysisTK.jar GenomeAnalysisTK.jar


## does it work?

reference="/Users/danthomas/Documents/naiandinus_gen/Luteus_genome/Mimulus_luteus.fasta"
vcf="/Users/danthomas/Documents/naiandinus_gen/Mll_x_MN.merged.ver0.1.vcf"
output="/Users/danthomas/Documents/naiandinus_gen/Mn_v0.1.fasta"

java -jar GenomeAnalysisTK.jar -T FastaAlternateReferenceMaker -R $reference -o $output -V $vcf

INFO  15:59:45,824 HelpFormatter - ------------------------------------------------------------------------------------ 
INFO  15:59:45,845 HelpFormatter - The Genome Analysis Toolkit (GATK) v3.8-1-0-gf15c1c3ef, Compiled 2018/02/19 05:43:50 
INFO  15:59:45,845 HelpFormatter - Copyright (c) 2010-2016 The Broad Institute 
INFO  15:59:45,845 HelpFormatter - For support and documentation go to https://software.broadinstitute.org/gatk 
INFO  15:59:45,845 HelpFormatter - [Wed May 02 15:59:45 PDT 2018] Executing on Mac OS X 10.12.6 x86_64 
INFO  15:59:45,845 HelpFormatter - Java HotSpot(TM) 64-Bit Server VM 1.8.0_144-b01 
INFO  15:59:45,849 HelpFormatter - Program Args: -T FastaAlternateReferenceMaker -R /Users/danthomas/Documents/naiandinus_gen/Luteus_genome/Mimulus_luteus.fasta -o /Users/danthomas/Documents/naiandinus_gen/Mn_v0.1.fasta -V /Users/danthomas/Documents/naiandinus_gen/Mll_x_MN.merged.ver0.1.vcf 
INFO  15:59:45,909 HelpFormatter - Executing as danthomas@wks-14-199 on Mac OS X 10.

## <a id="gencov">Making a map of genome coverage</a>

### <a id="bigCov">Make overall coverage map</a>

Since we are defaulting to a foreign genome when the information from our hybrid reads is lacking, we should be ready to check on the number reads ("read depth") we have for any region of interest. Let's go brute force, and make a BP-by-BP map. To do this we'll use yet another set of tools, the [bedtools suite](http://bedtools.readthedocs.io/en/latest/content/bedtools-suite.html), specifically the [genomecov](http://bedtools.readthedocs.io/en/latest/content/tools/genomecov.html) tool. 

In [ ]:
## shortcut to our BAM
mergedBAM=/Users/danthomas/Documents/naiandinus_gen/BAM/Mll_x_MN.merged.bam

## make the map
bedtools genomecov -ibam $mergedBAM -d > MllxMna_genome_cov_BP.txt

The result is a massive file, that gives the number of reads that cover each base pair:

In [19]:
ls -lh MllxMna_genome_cov_BP.txt 

-rw-r--r--  1 danthomas  staff    10G Apr 29 13:00 MllxMna_genome_cov_BP.txt


In [20]:
head -n 15 MllxMna_genome_cov_BP.txt 

lcl|scaffold_32	1	0
lcl|scaffold_32	2	1
lcl|scaffold_32	3	2
lcl|scaffold_32	4	2
lcl|scaffold_32	5	2
lcl|scaffold_32	6	4
lcl|scaffold_32	7	4
lcl|scaffold_32	8	4
lcl|scaffold_32	9	5
lcl|scaffold_32	10	6
lcl|scaffold_32	11	6
lcl|scaffold_32	12	7
lcl|scaffold_32	13	8
lcl|scaffold_32	14	10
lcl|scaffold_32	15	10


### <a id="subScaff">Subset coverage map to scaffolds of interest</a>

Since this is so massive, let's subset to just areas we're interested in. How can we pull out scaffolds we're interested in? Lately, we are most interested in the pla2 locus, which is on scaffold 26...let's pull that out.

In [21]:
## mac os has a weird word boundry character for regex:
sed -n '/scaffold_26[[:>:]]/p' MllxMna_genome_cov_BP.txt > scaffold_coverages/Mna_scaffold_26_coverage.tsv

In [24]:
cd /Users/danthomas/Documents/naiandinus_gen/scaffold_coverages

head Mna_scaffold_26_coverage.tsv
tail Mna_scaffold_26_coverage.tsv

lcl|scaffold_26	1	0
lcl|scaffold_26	2	0
lcl|scaffold_26	3	0
lcl|scaffold_26	4	0
lcl|scaffold_26	5	0
lcl|scaffold_26	6	0
lcl|scaffold_26	7	0
lcl|scaffold_26	8	0
lcl|scaffold_26	9	0
lcl|scaffold_26	10	0
lcl|scaffold_26	830788	5
lcl|scaffold_26	830789	5
lcl|scaffold_26	830790	5
lcl|scaffold_26	830791	2
lcl|scaffold_26	830792	2
lcl|scaffold_26	830793	1
lcl|scaffold_26	830794	1
lcl|scaffold_26	830795	0
lcl|scaffold_26	830796	0
lcl|scaffold_26	830797	0


## <a id="subGen">Subset <i>M. naiandinus</i> genome to just scaffolds of interest</a>

While we we're at it, let's pull out the sequences themselves from these areas of interest. For scaffold 26, for example. 

First we have to find the contig(s) in our new <i>M. naiandinus</i> genome that correspond to scaffold 26:

In [27]:
cd /Users/danthomas/Documents/naiandinus_gen
grep "scaffold_26:" Mn_v0.1.fasta

>21 lcl|scaffold_26:1


So contig 21 is our stand in for scaffold 26. Let's make a local [blast](https://blast.ncbi.nlm.nih.gov/Blast.cgi) database of our new genome, and pull out this contig. 

In [28]:
## make our database
makeblastdb -in Mn_v0.1.fasta -parse_seqids -dbtype nucl

## pull out contig 21 / scaffold 26
blastdbcmd -entry 21 -db Mn_v0.1.fasta | head

>21 lcl|scaffold_26:1
GTGCGGCTCAAGTTTACGCGCGGCTTTATGTTTCTCGCAACCCCTTGGTTCATAGTCCATCCTTGCACCAGTGTAAAGTC
TTACCTGTATAAGAAACCAATACATATATACTTGATTAAAATTATTATCCTTTTCCATTTTTCTTATTCCTGTCTCTTAT
ACACATCTAGATGTGTATAAGAGACAGGGTTCATAGTCCATCCTTGCACCAGTGTAAAGTCTTACCTGTATAAGAAACCA
ATACATATACTTGATTAAAATATTTATCCTTTCCATTTTTCTTATTCTCTTGAAGATTCTTGTTGTAATAATATCAAGAA
AAAAGAGAAGAACACTTACATTCTGAGGACCAATCAGCTCCAAGAGATGTCCGTCATCGTTTTTTGTGCTTGGAAGCTTA
AGGGACCATGGAGAATTTACTAGTTTCCATTCTGAATCTGCAAACTCAGCTAGTGTACACGTTTCCCCTGAATTAAAAAG
GCCAATCGCAGCTTTTCTTCTACATTCTTTCTTCCCCTTCGATCTCTCCAAATCCCTGTGAATGAGGCCATCCAAACACA
TTAAATTTGTTAAAACCAAAACCATCTTCAAGAAAATCCAAGGAAATATGTACACAAACTTCAAATACCTCATTTGTAGG
CTGAGCACTTGATTTCCATCCTCGTCAACCACACGAGTCCAACTCTTAGTAAATTGAGGCAGTGGGAACAAACAAGAACT


Looks right. Write it out.

In [ ]:
## pull out contig 21 / scaffold 26
blastdbcmd -entry 21 -db Mn_v0.1.fasta > Mna_scaffold_26.fasta

## <a id="spotcheck">Spot check: polymorphism in pla2</a>

Andreas and Arielle recently targeted the Myb3a gene of <i>M. naiandinus</i> using primers designed from <i>M. luteus</i>. They compared their resulting sanger sequences to the <i>M. naiandinus</i> draft genome and the <i>M. luteus</i> genome in the pla2 locus. They found a SNP between <i>M. luteus</i> and <i>M. naiandinus</i> not detected in the draft genome created above, at BP 147,678 in our scaffold 26. The <i>M. luteus</i> genome and draft <i>M. naiandinus</i> genome both have a G at this locus, but the sanger sequences of <i>M. naiandinus</i> show a T there:

![SNP](Andreas_SNP.png)

So how come our variant calling procedure didn't pick this up? We can look for this variant in our VCF...

In [33]:
## shortcut to vcf
vcf="Mll_x_MN.merged.ver0.1.vcf"

So let's get a regex that works...for example, our first variant:

In [37]:
grep "#CHROM" $vcf -A 2

#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	S16	S17	S18	S19	S20	S21
lcl|scaffold_32	209	.	G	A	210.56	.	AC=5;AF=0.417;AN=12;BaseQRankSum=0.340;ClippingRankSum=0.000;DP=20;ExcessHet=9.1645;FS=2.171;MLEAC=5;MLEAF=0.417;MQ=44.00;MQRankSum=0.000;QD=26.32;ReadPosRankSum=0.996;SOR=0.528	GT:AD:DP:GQ:PL	0/1:1,1:2:39:39,0,40	0/0:2,0:2:6:0,6,92	0/1:3,1:4:33:33,0,106	0/1:3,1:4:33:33,0,124	0/1:1,2:3:33:81,0,33	0/1:3,2:5:62:62,0,118
lcl|scaffold_32	340	.	T	C	285.29	.	AC=6;AF=0.600;AN=10;BaseQRankSum=0.728;ClippingRankSum=0.000;DP=15;ExcessHet=7.2016;FS=0.000;MLEAC=6;MLEAF=0.600;MQ=44.00;MQRankSum=0.000;QD=25.36;ReadPosRankSum=-0.642;SOR=1.061	GT:AD:DP:GQ:PL	1/1:0,2:2:6:90,6,0	0/1:2,1:3:36:36,0,81	./.	0/1:1,3:4:24:123,0,24	0/1:1,1:2:38:38,0,39	0/1:3,1:4:33:33,0,123


We can find like this:

In [38]:
grep scaffold_32\\t209\\t $vcf  ## nada

lcl|scaffold_32	209	.	G	A	210.56	.	AC=5;AF=0.417;AN=12;BaseQRankSum=0.340;ClippingRankSum=0.000;DP=20;ExcessHet=9.1645;FS=2.171;MLEAC=5;MLEAF=0.417;MQ=44.00;MQRankSum=0.000;QD=26.32;ReadPosRankSum=0.996;SOR=0.528	GT:AD:DP:GQ:PL	0/1:1,1:2:39:39,0,40	0/0:2,0:2:6:0,6,92	0/1:3,1:4:33:33,0,106	0/1:3,1:4:33:33,0,124	0/1:1,2:3:33:81,0,33	0/1:3,2:5:62:62,0,118


Now try this in the neighborhood of where we know the variant is:

In [39]:
grep scaffold_26\\t147[0-9][0-9][0-9]\\t $vcf

lcl|scaffold_26	147085	.	CA	C	85.43	.	AC=4;AF=0.500;AN=8;BaseQRankSum=0.000;ClippingRankSum=0.000;DP=7;ExcessHet=0.1902;FS=0.000;MLEAC=4;MLEAF=0.500;MQ=43.30;MQRankSum=-1.036;QD=33.27;ReadPosRankSum=0.253;SOR=2.780	GT:AD:DP:GQ:PL	./.	0/0:1,0:1:3:0,3,81	0/0:1,0:1:3:0,3,81	1/1:0,1:1:3:36,3,0	1/1:0,2:2:6:90,6,0	./.
lcl|scaffold_26	147099	.	T	A	60.01	.	AC=2;AF=0.333;AN=6;BaseQRankSum=-0.842;ClippingRankSum=0.000;DP=5;ExcessHet=0.4576;FS=0.000;MLEAC=2;MLEAF=0.333;MQ=43.02;MQRankSum=-1.645;QD=30.01;ReadPosRankSum=1.282;SOR=2.204	GT:AD:DP:GQ:PL	./.	0/0:2,0:2:6:0,6,126	0/0:1,0:1:3:0,3,81	./.	1/1:0,2:2:6:90,6,0	./.
lcl|scaffold_26	147792	.	C	T	132.53	.	AC=3;AF=0.300;AN=10;BaseQRankSum=0.431;ClippingRankSum=0.000;DP=12;ExcessHet=0.7918;FS=0.000;MLEAC=3;MLEAF=0.300;MQ=42.58;MQRankSum=-0.464;QD=26.51;ReadPosRankSum=0.731;SOR=0.169	GT:AD:DP:GQ:PL	0/0:1,0:1:3:0,3,37	0/0:5,0:5:15:0,15,216	1/1:0,2:2:6:90,6,0	0/0:1,0:1:3:0,3,45	./.	0/1:1,2:3:36:78,0,36
lcl|scaffold_26	147892	.	G	C	213.01	.	AC=5;AF=0.41

Nada. Why not? Well, part of the answer may be explained [here](https://gatkforums.broadinstitute.org/gatk/discussion/6005/allele-depth-ad-is-lower-than-expected). The haplotypecaller has some strict cutoffs for calling differences as true variants. Its algorithms take read depth, base call qualities, and mapq qualities (the confidence in the alignment of a given basepair), and decides whether to consider a read informational. 

We can look at our very first variant call again as an example:

In [40]:
grep "#CHROM" $vcf -A 1

#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	S16	S17	S18	S19	S20	S21
lcl|scaffold_32	209	.	G	A	210.56	.	AC=5;AF=0.417;AN=12;BaseQRankSum=0.340;ClippingRankSum=0.000;DP=20;ExcessHet=9.1645;FS=2.171;MLEAC=5;MLEAF=0.417;MQ=44.00;MQRankSum=0.000;QD=26.32;ReadPosRankSum=0.996;SOR=0.528	GT:AD:DP:GQ:PL	0/1:1,1:2:39:39,0,40	0/0:2,0:2:6:0,6,92	0/1:3,1:4:33:33,0,106	0/1:3,1:4:33:33,0,124	0/1:1,2:3:33:81,0,33	0/1:3,2:5:62:62,0,118


5 of our 6 read groups have this site as heterozygous G/A. If we sum up, 20 reads were considered informational among the various read groups. So this site had pretty deep coverage, and just a few reads that were not considered useful. What was our coverage at the variant call site?

In [48]:
scafcov=vis_cov/MllxMna_genome_cov_BP.txt
grep scaffold_32\\t209\\t $scafcov -A 5 -B 5

lcl|scaffold_32	204	27
lcl|scaffold_32	205	25
lcl|scaffold_32	206	24
lcl|scaffold_32	207	23
lcl|scaffold_32	208	23
lcl|scaffold_32	209	22
lcl|scaffold_32	210	21
lcl|scaffold_32	211	20
lcl|scaffold_32	212	20
lcl|scaffold_32	213	20
lcl|scaffold_32	214	20



Looks like there was some loss of information, 22 reads covered the site but only 20 were informational.

And what was our coverage at Andreas's SNP site in pla2?

In [45]:
scaf26cov=/Users/danthomas/Documents/naiandinus_gen/scaffold_coverages/Mna_scaffold_26_coverage.tsv
grep scaffold_26\\t147678\\t $scaf26cov -A 5 -B 5

lcl|scaffold_26	147673	8
lcl|scaffold_26	147674	8
lcl|scaffold_26	147675	7
lcl|scaffold_26	147676	7
lcl|scaffold_26	147677	7
lcl|scaffold_26	147678	7
lcl|scaffold_26	147679	7
lcl|scaffold_26	147680	7
lcl|scaffold_26	147681	8
lcl|scaffold_26	147682	8
lcl|scaffold_26	147683	8


My best guess at this point is that we lacked the coverage (7 reads) and maybe the quality to give the haplotyper the confidence to call many of the variants. But as Josh has pointed out, we haven't actually looked at those reads that supposedly cover this area... samtools let us take a peek into the merged BAM file that we have, so we can see these reads:

In [6]:
cd /Users/danthomas/Documents/naiandinus_gen/BAM
samtools view Mll_x_MN.merged.bam  "lcl|scaffold_26:147676-147682" 

K00282:27:HC3GVBBXX:2:1109:1986:27971	16	lcl|scaffold_26	147639	44	51M	*	0	0	GCTACAGATAATGTCGAGTACTGCAATGAAAATCACATGTTCGAAAAGAAC	JJJFJFJJJJFJJFJJJJJJFJJFJJJJFJJJJJJJJJJJFFJJJJFFAAA	MD:Z:51	XG:i:0	NM:i:0	XM:i:0	XN:i:0	XO:i:0	AS:i:102	YT:Z:UU	RG:Z:27-7026D542	PG:Z:MarkDuplicates-203924BB
K00282:27:HC3GVBBXX:2:2225:8156:21887	16	lcl|scaffold_26	147641	44	51M	*	0	0	TACAGATAATGTCGAGTACTGCAATGAAAATCACATGTTCGAAAAGAACCC	JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJFJJJJJJJJJJJFFFAA	MD:Z:51	XG:i:0	NM:i:0	XM:i:0	XN:i:0	XO:i:0	AS:i:102	YT:Z:UU	RG:Z:27-7B4F811C	PG:Z:MarkDuplicates-26667DE3
K00282:27:HC3GVBBXX:2:2223:27874:28692	16	lcl|scaffold_26	147644	44	51M	*	0	0	AGATAATGTCGAGTACTGCAATGAAAATCACATGTTCGAAAAGAACCCATC	JJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJJFFFAA	MD:Z:51	XG:i:0	NM:i:0	XM:i:0	XN:i:0	XO:i:0	AS:i:102	YT:Z:UU	RG:Z:27-4A2064A6	PG:Z:MarkDuplicates-5693B18F
K00282:27:HC3GVBBXX:2:1205:4980:35901	16	lcl|scaffold_26	147651	44	51M	*	0	0	GTCGAGTACTGCAATGAAAATCACATGTTCGAAAAGAACCCATCTGATTTC	JJJJJJJJJ

This generally agrees with our coverage map. But where is our SNP in these? Cut this down to just the sequences, and grep out the SNP region:

In [7]:
cd /Users/danthomas/Documents/naiandinus_gen/BAM
samtools view Mll_x_MN.merged.bam  "lcl|scaffold_26:147676-147682" | cut -f 10 | grep -E "TTG.CTG|$"

GCTACAGATAATGTCGAGTACTGCAATGAAAATCACATGTTCGAAAAGAAC
TACAGATAATGTCGAGTACTGCAATGAAAATCACATGTTCGAAAAGAACCC
AGATAATGTCGAGTACTGCAATGAAAATCACATGTTCGAAAAGAACCCATC
GTCGAGTACTGCAATGAAAATCACATGTTCGAAAAGAACCCATCTGATTTC
CAATGAAAATCACATGTTCGAAAAGAACCCATCTGATTTCTTATTGTCTGG
ATGAAAATCACATGTTCGAAAAGAACCCATCTGATTTCTTATTGGCTGGGC
ATCACATGTTCGAAAAGAACCCATCTGATTTCTTATTGGCTGGGCCAATTA
GAAAAGAACCCATCTGATTTCTTATTGGCTGGGCCAATTAACGATCATTAT


Huh, only four of our reads seem to have that locus in them, really. And of these, have the "T" base pair that we found in Andreas' sequence. 
